In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
import lxml
import os
from time import sleep

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) 
# and collect the latest News Title and Paragraph Text. 

url = 'https://mars.nasa.gov/news/'
driver = webdriver.Chrome()
driver.get(url)
# Create BeautifulSoup object
soup = bs(driver.page_source, 'html.parser')

In [4]:
news_title = soup.find('div', class_='content_title').text
print(f"Most Recent Title: {news_title}")
news_p = soup.find('div', class_='article_teaser_body').text
print(f"First Paragraph: {news_p}")
driver.close()

Most Recent Title: Scientists Explore Outback as Testbed for Mars 
First Paragraph: Australia provides a great place for NASA's Mars 2020 and the ESA-Roscosmos ExoMars scientists to hone techniques in preparation for searching for signs ancient life on Mars.


In [5]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
# assign the url string to a variable called `featured_image_url`.

In [6]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
# Navigate to the page with the full, large image
browser.click_link_by_id('full_image')
sleep(5)
browser.click_link_by_partial_text('more info')

In [8]:
html = browser.html
soup = bs(html, 'html.parser')
image_url = soup.find('img').get('src')
base_url = "https://www.jpl.nasa.gov/spaceimages"
featured_img_url = base_url + image_url
print (f"The featured image's URL is: {featured_img_url}")

The featured image's URL is: https://www.jpl.nasa.gov/spaceimages/assets/images/logo_nasa_trio_black@2x.png


In [9]:
### Mars Weather
# Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) 
# Scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called `mars_weather`.

In [10]:
twitter_wx_url = "https://twitter.com/marswxreport?lang=en"
tweet_response = requests.get(twitter_wx_url)
tweet_soup = bs(tweet_response.text, 'html.parser')
mars_weather = tweet_soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print("Today's weather on Mars:")
print(mars_weather)

Today's weather on Mars:
InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)
winds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)
pressure at 7.70 hPapic.twitter.com/MhPPOHJg3m


In [11]:
### Mars Facts

# Visit the Mars Facts webpage [here](http://space-facts.com/mars/) 
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

mars_facts_url = "http://space-facts.com/mars/"
sleep(3)  
mars_table = pd.read_html(mars_facts_url) 
mars_facts_df = mars_table[0]
mars_facts_df

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-153 to 20 °C,-88 to 58°C


In [12]:
# Use Pandas to convert the data to a HTML table string.
html_table = mars_facts_df.to_html()
print(html_table.translate({ord('\n'): None}))

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>1</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>2</th>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>3</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>4</th>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>5</th>      <td>Temperature:</td>      <td>-153 to 20 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>


In [13]:
### Mars Hemispheres

# Visit the USGS Astrogeology site [here]:
# (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) 
# to obtain high resolution images for each of Mar's hemispheres.

In [14]:
base_url = "https://astrogeology.usgs.gov/"
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemi_url)
hemi_html = browser.html
hemi_soup = bs(hemi_html, 'html.parser')

In [15]:
# Use a Python dictionary to store the data using the keys `img_url` and `title`.
# Append the dictionary with the image url string and the hemisphere title to a list. 
# This list will contain one dictionary for each hemisphere.

In [16]:
hemi_dict_list = []
items = hemi_soup.findAll('div', class_='item')
# Save both the image url string for the full resolution hemisphere image, 
# and the Hemisphere title containing the hemisphere name. 
for item in items:
    title = item.find('h3').text
    # Click each of the links to the hemispheres in order to find the image url to the full resolution image.
    browser.click_link_by_partial_text('Hemisphere Enhanced')
    browser.windows.current
    hemi_img_soup = bs(browser.html, 'html.parser')
    hemi_img_full = hemi_img_soup.find('div', class_='downloads')
    hemi_img_url = hemi_img_full.find('a').get('href')
    hemi_dict = {'title':title , 'image url': hemi_img_url}
    hemi_dict_list.append(hemi_dict)
browser.quit()
hemi_dict_list  

[{'title': 'Cerberus Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]